In [22]:
#import libraries  
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
#Tensorflow is a google AI framework
import tensorflow as tf
#Keras is a DeepLearning API for Tensorflow
from tensorflow import keras
from tensorflow.keras import layers

In [23]:
#Import dataset
column_names = ['FILA','PLANTA','CLL','NDVI','CVI','GNDVI','NRVI','RVI','GCI']
raw_dataset = pd.read_csv('./processDfClo.csv',names=column_names,
                          na_values = "?", sep=",", skipinitialspace=True)
dataset = raw_dataset.copy()
dataset.head()

,FILA,PLANTA,CLL,NDVI,CVI,GNDVI,NRVI,RVI,GCI
0,1,2,114.51,0.227151,14.582.108.702.048.800,0.519744,0.227151,15.894.613.401.746.400,15.198.877.138.937.700
1,2,4,111.03,0.232475,15.048.080.329.891.500,0.550758,0.232475,16.071.226.908.890.300,15.508.927.545.921.700
2,3,6,117.61,0.227241,14.336.495.612.171.100,0.505362,0.227241,15.896.344.631.347.700,15.055.035.447.449.000
3,4,9,103.48,0.229360,14.456.483.229.422.500,0.515255,0.229360,15.965.638.577.924.600,15.153.916.743.185.500
4,5,10,132.56,0.228474,15.153.168.177.586.800,0.550987,0.228474,15.937.060.749.703.400,15.511.430.409.115.100


In [24]:
#Verify NaN data
dataset.isna().sum()

FILA      0
PLANTA    0
CLL       0
NDVI      0
CVI       0
GNDVI     0
NRVI      0
RVI       0
GCI       0
dtype: int64

In [38]:
#Clean data (in this case is optional XD)
dataset = dataset.dropna()

In [43]:
#Divide th dataset into training and tests (80% for training process and 20% for evaluation)
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)
test_dataset.tail()

,FILA,PLANTA,CLL,NDVI,CVI,GNDVI,NRVI,RVI,GCI
39,11,22,135.41,0.238489,1.556.180.885.258.600,0.534050,0.238489,16.278.377.236.907.800,1.534.240.783.202.650
44,15,2,117.81,0.226692,19.995.533.301.376.000,0.480380,0.226692,15.875.220.069.358.300,14.805.352.099.011.800
47,12,9,128.44,0.233010,23.983.997.066.178.900,0.565249,0.233010,16.083.271.712.054.500,15.653.981.663.919.400
53,6,22,97.06,0.228409,25.836.587.081.574.100,0.485881,0.228409,15.937.387.270.071.500,14.860.165.431.431.100
55,4,26,97.43,0.236460,13.990.365.178.575.100,0.582689,0.236460,16.205.818.780.686.400,15.828.863.000.948.100


In [46]:
#View general data about train_data_set
train_stats = train_dataset.describe()
train_stats.pop("CLL")
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
FILA,46.0,8.130435,4.455253,1.000000,4.250000,8.000000,12.000000,15.000000
PLANTA,46.0,15.108696,8.329941,2.000000,9.000000,15.000000,22.750000,30.000000
NDVI,46.0,0.230437,0.004454,0.212748,0.227669,0.231061,0.232894,0.238740
GNDVI,46.0,0.530468,0.046508,0.390106,0.497548,0.536801,0.563785,0.615024
NRVI,46.0,0.230437,0.004454,0.212748,0.227669,0.231061,0.232894,0.238740


In [51]:
#Select the target label
train_labels = train_dataset.pop('CLL')
test_labels = test_dataset.pop('CLL')
train_labels.head()

35    114.58
34    140.40
42    147.67
27    135.05
11    121.15
Name: CLL, dtype: float64

In [52]:
#Build a model -- MLP secuential model with 2 layers
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model


In [53]:
#Build
model = build_model()

In [54]:
#Resume 
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 64)                576       
_________________________________________________________________
dense_7 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 65        
Total params: 4,801
Trainable params: 4,801
Non-trainable params: 0
_________________________________________________________________


In [55]:
train_dataset= train_dataset.astype('float32')
train_labels = train_labels.astype('float32')
train_dataset.head()

ValueError: could not convert string to float: '24.207.912.071.341.300'

In [14]:

#Train model
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 1000

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])



....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
..........................................................................................

In [16]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

,loss,mae,mse,val_loss,val_mae,val_mse,epoch
995,NaN,NaN,NaN,NaN,NaN,NaN,995
996,NaN,NaN,NaN,NaN,NaN,NaN,996
997,NaN,NaN,NaN,NaN,NaN,NaN,997
998,NaN,NaN,NaN,NaN,NaN,NaN,998
999,NaN,NaN,NaN,NaN,NaN,NaN,999
